In [1]:
import os
import requests
import json
from langchain.llms import AzureOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import load_tools, initialize_agent, create_sql_agent, AgentType
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.memory import ConversationBufferWindowMemory
from langchain.sql_database import SQLDatabase

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ["OPENAI_API_BASE"] = "https://qucy-openai-test.openai.azure.com/"
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"
os.environ["SERPAPI_API_KEY"] = os.getenv('SERPAPI_API_KEY')


# create large language model service
llm = AzureOpenAI(deployment_name="text-davinci-003", model_name="text-davinci-003")

In [2]:
llm("Hey what's up ?")

' \n\nHey there! Not much, just chillin. How about you?'

## 1. Use GPT+ Agent to query customer data (transaction data)

### Given below transaction table, we can use natural language to query below table

<img src="./credit_card_transaction_table.png" alt="Credit Card Transaction Table" width="1800"/>

In [10]:
db = SQLDatabase.from_uri("sqlite:////mnt/disk1/python-projects/LangChainDemo/demo.db")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

# create agent
database_agent = create_sql_agent(llm=llm, toolkit=toolkit, verbose=True)

# run agent
database_agent.run("What is my latest available credit limit ?")



> Entering new AgentExecutor chain...
Action: list_tables_sql_db
Action Input: 
Observation: sqlite_sequence, credit_card_transactions, stocks
Thought: I should check the schema of the credit_card_transactions table to see what columns it has.
Action: schema_sql_db
Action Input: credit_card_transactions
Observation: 
CREATE TABLE credit_card_transactions (
	id INTEGER, 
	transaction_date DATE NOT NULL, 
	category VARCHAR(255) NOT NULL, 
	description VARCHAR(255), 
	amount FLOAT NOT NULL, 
	transaction_type VARCHAR(255) NOT NULL, 
	card_number VARCHAR(16) NOT NULL, 
	credit_limit FLOAT NOT NULL, 
	available_credit FLOAT NOT NULL, 
	due_date DATE NOT NULL, 
	rewards_points INTEGER NOT NULL, 
	interest_rate FLOAT NOT NULL, 
	fee FLOAT NOT NULL, 
	PRIMARY KEY (id)
)

/*
3 rows from credit_card_transactions table:
id	transaction_date	category	description	amount	transaction_type	card_number	credit_limit	available_credit	due_date	rewards_points	interest_rate	fee
1	2023-04-01	groceries	Groce

'Your latest available credit limit is 1150.0.'

In [11]:
database_agent.run("What was my total spending in 2023 April ?")



> Entering new AgentExecutor chain...
Action: list_tables_sql_db
Action Input: ""
Observation: sqlite_sequence, credit_card_transactions, stocks
Thought: I need to look at the credit_card_transactions table.
Action: schema_sql_db
Action Input: "credit_card_transactions"
Observation: 
CREATE TABLE credit_card_transactions (
	id INTEGER, 
	transaction_date DATE NOT NULL, 
	category VARCHAR(255) NOT NULL, 
	description VARCHAR(255), 
	amount FLOAT NOT NULL, 
	transaction_type VARCHAR(255) NOT NULL, 
	card_number VARCHAR(16) NOT NULL, 
	credit_limit FLOAT NOT NULL, 
	available_credit FLOAT NOT NULL, 
	due_date DATE NOT NULL, 
	rewards_points INTEGER NOT NULL, 
	interest_rate FLOAT NOT NULL, 
	fee FLOAT NOT NULL, 
	PRIMARY KEY (id)
)

/*
3 rows from credit_card_transactions table:
id	transaction_date	category	description	amount	transaction_type	card_number	credit_limit	available_credit	due_date	rewards_points	interest_rate	fee
1	2023-04-01	groceries	Grocery Store	50.0000000000	purchase	12

'Your total spending in April 2023 was 550.0.'

In [12]:
database_agent.run("Can you show me a breakdown of my credit card by category ?")



> Entering new AgentExecutor chain...
Action: list_tables_sql_db
Action Input: ""
Observation: sqlite_sequence, credit_card_transactions, stocks
Thought: I need to look at the schema of the credit_card_transactions table to see what columns I can use in my query
Action: schema_sql_db
Action Input: "credit_card_transactions"
Observation: 
CREATE TABLE credit_card_transactions (
	id INTEGER, 
	transaction_date DATE NOT NULL, 
	category VARCHAR(255) NOT NULL, 
	description VARCHAR(255), 
	amount FLOAT NOT NULL, 
	transaction_type VARCHAR(255) NOT NULL, 
	card_number VARCHAR(16) NOT NULL, 
	credit_limit FLOAT NOT NULL, 
	available_credit FLOAT NOT NULL, 
	due_date DATE NOT NULL, 
	rewards_points INTEGER NOT NULL, 
	interest_rate FLOAT NOT NULL, 
	fee FLOAT NOT NULL, 
	PRIMARY KEY (id)
)

/*
3 rows from credit_card_transactions table:
id	transaction_date	category	description	amount	transaction_type	card_number	credit_limit	available_credit	due_date	rewards_points	interest_rate	fee
1	2023-

'The breakdown of your credit card by category is: payment (200.0), disputed charge (150.0), suspicious activity (100.0), groceries (50.0), gas (30.0), dining out (20.0), rewards (0.0) and interest (0.0).'

In [13]:
database_agent.run("What were the fees or interest charged on my credit card on 2023 April ?")



> Entering new AgentExecutor chain...
Action: list_tables_sql_db
Action Input: ""
Observation: sqlite_sequence, credit_card_transactions, stocks
Thought: I need to see the schema of the credit_card_transactions table to know what I can query for
Action: schema_sql_db
Action Input: "credit_card_transactions"
Observation: 
CREATE TABLE credit_card_transactions (
	id INTEGER, 
	transaction_date DATE NOT NULL, 
	category VARCHAR(255) NOT NULL, 
	description VARCHAR(255), 
	amount FLOAT NOT NULL, 
	transaction_type VARCHAR(255) NOT NULL, 
	card_number VARCHAR(16) NOT NULL, 
	credit_limit FLOAT NOT NULL, 
	available_credit FLOAT NOT NULL, 
	due_date DATE NOT NULL, 
	rewards_points INTEGER NOT NULL, 
	interest_rate FLOAT NOT NULL, 
	fee FLOAT NOT NULL, 
	PRIMARY KEY (id)
)

/*
3 rows from credit_card_transactions table:
id	transaction_date	category	description	amount	transaction_type	card_number	credit_limit	available_credit	due_date	rewards_points	interest_rate	fee
1	2023-04-01	groceries	G

'On 2023 April, the fees or interest charged on my credit card were 0.0 for the majority of the month, with the exception of the 15th, when the interest rate was 18.0.'

## 2. Use GPT+Agent to query latest financial market data (public stock price /other foreign exchange info in real-time)

In [58]:
tools = load_tools(["serpapi"], llm=llm)

search_agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, max_iterations=3)

In [56]:
search_agent.run("Apple Inc today's stock price ?")



> Entering new AgentExecutor chain...
 I need to find out the current stock price of Apple Inc
Action: Search
Action Input: "Apple Inc stock price today"
Observation: Apple Inc. is an American multinational technology company headquartered in Cupertino, California. Apple is the world's largest technology company by revenue, with US$394.3 billion in 2022 revenue. As of March 2023, Apple is the world's biggest company by market capitalization.
Thought: I can find the current stock price of Apple Inc on a financial website
Action: Search
Action Input: "Apple Inc stock price today" financial website
Observation: Stock Quote: NASDAQ: AAPL ; Day's Open173.16 ; Closing Price172.07 ; Volume37.2 ; Intraday High173.21 ; Intraday Low171.47.
Thought: I now know the final answer
Final Answer: As of March 2023, Apple Inc's stock price is 172.07.

> Finished chain.


"As of March 2023, Apple Inc's stock price is 172.07."

In [57]:
search_agent.run("HKD & USD exchange rate")



> Entering new AgentExecutor chain...
 I need to find out what the exchange rate is between HKD and USD
Action: Search
Action Input: HKD & USD exchange rate
Observation: Convert Hong Kong Dollar to US Dollar ; 1 HKD, 0.127607 USD ; 5 HKD, 0.638034 USD ; 10 HKD, 1.27607 USD ; 25 HKD, 3.19017 USD.
Thought: I now know the exchange rate between HKD and USD
Final Answer: 1 HKD = 0.127607 USD.

> Finished chain.


'1 HKD = 0.127607 USD.'

## 3. Use GPT + Agent to query financial news

### 3-1 create customized news retrieval tool

In [33]:
from langchain.tools import BaseTool

class NewsRetrievalTool(BaseTool):
    
    # set the proxy server address
    proxy_address = 'http://127.0.0.1:7890'

    # set the environment variables
    os.environ['http_proxy'] = proxy_address
    os.environ['https_proxy'] = proxy_address
    
    name = "News Retrieval Tool"
    description = "useful when you need to retrieve latest news"
    
    def _run(self, query:str):
        url = (
        'https://newsapi.org/v2/everything?'
        f'q={query}&'
       'from=2023-05-16&'
       'pageSize=3&'
       'sortBy=popularity&'
       'apiKey=')
        r = requests.get(url)
        json_obj = json.loads(json.dumps(r.json()))
        return str(json_obj['articles'])
    
    def _arun(self, query: str):
        raise NotImplementedError("Async operation not supported yet.")
        
        
tools = [NewsRetrievalTool()]

### 3-2 Create chat-conversational-react-description agent

In [50]:
agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=ConversationBufferWindowMemory(
        memory_key='chat_history',
        k=5,
        return_messages=True
    )
)

In [51]:
agent("Can you show me recent financial news ?")



> Entering new AgentExecutor chain...


RESPONSE
--------------------
```json
{
    "action": "News Retrieval Tool",
    "action_input": "Financial News"
}
```
Observation: [{'source': {'id': 'the-washington-post', 'name': 'The Washington Post'}, 'author': None, 'title': "Officials say social media is hurting teens. Scientists say it's complicated...", 'description': "Officials say social media is hurting teens. Scientists say it's complicated...\r\n\n \n \n \n (Third column, 9th story, link)\r\n\n \r\n\n \r\n\n \n \n Drudge Report Feed needs your support!\xa0\xa0 Become a Patron", 'url': 'https://www.washingtonpost.com/politics/2023/05/10/officials-say-social-media-is-hurting-teens-scientists-say-its-complicated/', 'urlToImage': 'https://img-s-msn-com.akamaized.net/tenant/amp/entityid/AA1b0cY2.img?h=630&w=1200&m=6&q=60&o=t&l=f&f=jpg&x=474&y=115', 'publishedAt': '2023-05-16T01:10:38Z', 'content': "Happy Wednesday! I still can't get over this story on the price of the modern car. Send

{'input': 'Can you show me recent financial news ?',
 'chat_history': [],
 'output': 'Based on the news retrieval tool, the latest financial news is that TPG has grown its credit business with a $2.7 billion acquisition from Angelo Gordon.'}

In [52]:
agent("Can you show me recent sports news ?")



> Entering new AgentExecutor chain...


RESPONSE
--------------------
```json
{
    "action": "News Retrieval Tool",
    "action_input": "Sports"
}
```
Observation: [{'source': {'id': None, 'name': 'Osxdaily.com'}, 'author': 'Jamie Cuevas', 'title': 'Second Release Candidate of iOS 16.5, iPadOS 16.5, MacOS Ventura 13.4 Available for Download', 'description': 'A second Release Candidate build of iOS 16.5, iPadOS 16.5, and MacOS Ventura 13.4 has been released by Apple to users participating in the public beta and developer beta programs for Apple system software. Apple had previously stated that final versions of iO…', 'url': 'https://osxdaily.com/2023/05/15/second-release-candidate-of-ios-16-5-ipados-16-5-macos-ventura-13-4-available-for-download/', 'urlToImage': 'https://cdn.osxdaily.com/wp-content/uploads/2022/07/public-beta-macos-ventura-ios-16-ipados-16.jpg', 'publishedAt': '2023-05-16T00:34:38Z', 'content': 'A second Release Candidate build of iOS 16.5, iPadOS 16.5, and MacOS Ven

{'input': 'Can you show me recent sports news ?',
 'chat_history': [HumanMessage(content='Can you show me recent financial news ?', additional_kwargs={}, example=False),
  AIMessage(content='Based on the news retrieval tool, the latest financial news is that TPG has grown its credit business with a $2.7 billion acquisition from Angelo Gordon.', additional_kwargs={}, example=False)],
 'output': 'The latest news related to sports is that Telly, a startup co-founded by Pluto co-founder Ilya Pozin, is offering 500,000 dual-screen TVs to those who sign up and provide information on their preferences and interests. Additionally, a second Release Candidate build of iOS 16.5, iPadOS 16.5, and MacOS Ventura 13.4 has been released by Apple to users participating in the public beta and developer beta programs for Apple system software. Lastly, a body of a missing 4-year-old boy has been found on a shoreline in Massachusetts.'}